In [ ]:
# !pip install tensorflow-gpu==1.15
!pip install ../input/wheatlibraries/gast-0.2.2/gast-0.2.2

In [ ]:
!pip install ../input/mytf115/astor-0.8.1-py2.py3-none-any.whl

In [ ]:
!pip install ../input/wheatlibraries/Keras_Applications-1.0.8-py3-none-any.whl

In [ ]:
!pip install ../input/tensorflowv1/tensorboard-1.15.0-py3-none-any.whl

In [ ]:
!pip install ../input/tensorflowv1/tensorflow_estimator-1.15.1-py2.py3-none-any.whl

In [ ]:
!pip install ../input/mytf115/tensorflow_gpu-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl

In [ ]:
!pip install ../input/wheatlibraries/Keras-2.2.4-py2.py3-none-any.whl

In [ ]:
!cp -Rf ../input/wheatprediction/WheatPrediction/yolo3 ./
!mkdir ./predicted

In [ ]:
import colorsys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import cv2


import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import image_preporcess

class YOLO(object):
    _defaults = {
        "model_path": '../input/wheatprediction/WheatPrediction/logs/trained_weights_final.h5',
        "anchors_path": '../input/wheatprediction/WheatPrediction/model_data/yolo_anchors.txt',
        "classes_path": '../input/wheatprediction/WheatPrediction/4_CLASS_test_classes.txt',
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "text_size" : 3,
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))

        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = image_preporcess(np.copy(image), tuple(reversed(self.model_image_size)))
            image_data = boxed_image

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.shape[0], image.shape[1]],#[image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        #print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        thickness = (image.shape[0] + image.shape[1]) // 600
        fontScale=1
        ObjectsList = []
        
        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)
            #label = '{}'.format(predicted_class)
            scores = '{:.2f}'.format(score)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.shape[0], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.shape[1], np.floor(right + 0.5).astype('int32'))

            mid_h = (bottom-top)/2+top
            mid_v = (right-left)/2+left

            # put object rectangle
            cv2.rectangle(image, (left, top), (right, bottom), self.colors[c], thickness)

            # get text size
            (test_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, thickness/self.text_size, 1)

            # put text rectangle
            cv2.rectangle(image, (left, top), (left + test_width, top - text_height - baseline), self.colors[c], thickness=cv2.FILLED)

            # put text above rectangle
            cv2.putText(image, label, (left, top-2), cv2.FONT_HERSHEY_SIMPLEX, thickness/self.text_size, (0, 0, 0), 1)

            # add everything to list
            ObjectsList.append([top, left, bottom, right, mid_v, mid_h, label, scores])

        return image, ObjectsList

    def close_session(self):
        self.sess.close()

    def detect_img(self, image):
        image = cv2.imread(image, cv2.IMREAD_COLOR)
        original_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_image_color = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        
        r_image, ObjectsList = self.detect_image(original_image_color)
        return r_image, ObjectsList

    


In [ ]:
yolo = YOLO()
# image = '../input/global-wheat-detection/test/2fd875eaa.jpg'
# r_image, ObjectsList = yolo.detect_img(image)

# cv2.imwrite('predicted/2fd875eaa_prediction.jpg', r_image)


In [ ]:
# def imShow(path):
#   import cv2
#   import matplotlib.pyplot as plt
#   %matplotlib inline

#   image = cv2.imread(path)
#   height, width = image.shape[:2]
#   resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

#   fig = plt.gcf()
#   fig.set_size_inches(18, 10)
#   plt.axis("off")
#   plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
#   plt.show()


In [ ]:
# imShow('predicted/2fd875eaa_prediction.jpg')

In [ ]:
#top, left, bottom, right, mid_v, mid_h, label, scores
# ObjectsList

In [ ]:
import pandas as pd

In [ ]:
submission_df = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')

In [ ]:
submission_df.columns

In [ ]:
for index, row in submission_df.iterrows():
    print(row['image_id'])
    image_name = '../input/global-wheat-detection/test/'+row['image_id']+'.jpg'
    r_image, ObjectsList = yolo.detect_img(image_name)
#     print(ObjectsList)
    ObjectsDictList=[]
    for prediction in ObjectsList:
        predictionDict = {}
        #top, left, bottom, right, mid_v, mid_h, label, scores
        predictionDict["confidence"]=prediction[7]
        predictionDict["xmin"]=prediction[1]
        predictionDict["ymin"]=prediction[0]
        predictionDict["width"]=prediction[3]-prediction[1]
        predictionDict["height"]=prediction[2]-prediction[0]
        ObjectsDictList.append(predictionDict)
    
    ObjectsDictList=sorted(ObjectsDictList, key = lambda i: i['confidence'],reverse=True)
    predictionString=""
    
    for ObjectsDict in ObjectsDictList:
        predictionString+=(str(ObjectsDict['confidence'])+" "
                           +str(ObjectsDict["xmin"])+" "
                           +str(ObjectsDict["ymin"])+" "
                           +str(ObjectsDict["width"])+" "
                           +str(ObjectsDict["height"])+" ")
    
    predictionString=predictionString.strip()
    
    if(len(predictionString)!=0):
        submission_df.loc[index,"PredictionString"]=predictionString

In [ ]:
submission_df.to_csv("submission.csv",index=False)